# Lab 02

In [25]:
import numpy as np

### Turn the dataset into a dataset compatible with Fastext

In [5]:
from datasets import load_dataset

In [16]:
import string
def process(txt: str) -> str:
    """
    Converts all uppercase letters to lowercase, replaces all punctuation marks with spaces, and returns the processed string.
    """
    
    lowercase_txt = txt.lower()
    txt.replace('<br>', ' ').replace('\n', ' ').strip()
    # create a translation table using maketrans method
    replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    
    # use the translate method to replace the punctuation
    processed_txt = lowercase_txt.translate(replace_punctuation)
    return processed_txt
process("What's your name? I'm Ba-yes")

'what s your name  i m ba yes'

In [17]:
dataset = load_dataset("imdb")

with open('train.txt', 'w', encoding='utf-8') as f:
    for example in dataset['train'].shuffle():
        text = example['text'].replace('\n', ' ').strip()
        label = '__label__' + ('positive' if example['label'] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

with open('test.txt', 'w', encoding='utf-8') as f:
    for example in dataset['test'].shuffle():
        text = example['text'].replace('\n', ' ').strip()
        label = '__label__' + ('positive' if example['label'] else 'negative')
        f.write(label + ' ' + process(text) + '\n')

f.close()

with open('train.txt', 'r') as f:
    first_line = f.readline()
    print(first_line)
f.close()

Found cached dataset imdb (C:/Users/antho/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

__label__negative how this movie got made with a supposedly  70 million budget and without being completely retooled is beyond me  the storyline and dialogue are beyond amateurish  characters say things no real person would ever say and almost never react to things that were said before  no one seems to be grounded in the real world  the acting of the leads is fine given that the writing is such a dud   but several actors in supporting roles really drag the production down  the hero s hair probably should ve gotten its own credit  it was so oddly attention  grabbing   not to mention that it gave one of the better performances in the pic  finally  for a movie about l a  being besieged by giant reptiles  this film is shockingly boring  what a shame  if you do see this  your mind will be constantly racing  thinking up ways that you could have taken the sfx scenes and built a far better movie around them  sadly  it wouldn t have taken much 



### Train a FastText classifier with default parameters on the training data, and evaluate it on the test data

In [22]:
import fasttext
model = fasttext.train_supervised(input="train.txt")

(('__label__negative',), array([1.00001001]))

In [58]:
preds, _ = model.predict(list(map(process, dataset['test']['text'])))
pred_labels = []
for pred in preds:
    if pred == ['__label__negative']:
        pred_labels.append(0)
    else:
        pred_labels.append(1)
pred_labels
accuracy = np.mean([dataset['test']['label'][i] == pred_labels[i] for i in range(len(pred_labels))])

KeyboardInterrupt: 